In [11]:
# Load in DropletUtils - empty droplet detection
# Load in scds - doublet detection
# Load in scran - normalization 

suppressPackageStartupMessages(library(DropletUtils))
suppressPackageStartupMessages(library(scds))
library(scran)
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(data.table))

In [17]:
run_qc <- function(file,name,replicate,tissue,outdir) {
    # Read in CellRanger cell x gene matrix (unfiltered)
    mtx = read10xCounts(file)
    ###### EMPTY DROPLET REMOVAL ######
    all_cells = counts(mtx)
    # Identify empty droplets
    cells_called = defaultDrops(all_cells,lower.prop=0.05)
    # Remove empty droplets
    real_cells  = all_cells[,cells_called]
    print("Starting droplet count")
    print(ncol(all_cells))
    print("Cells called after removing empty droplets")
    print(ncol(real_cells))
    ###### DOUBLET REMOVAL ######
    # load filtered cells into single cell experiment
    sce = SingleCellExperiment(list(counts=real_cells))
    # score singlets/doublets
    sce_cell = cxds_bcds_hybrid(sce)
    # identify singlets
    singlets = colData(sce_cell)$hybrid_score<1.3
    # only keep singlets (remove doublets)
    singlet.counts =   counts(sce_cell)[, singlets]
    # save data back to single cell experiment
    sce_singlet = SingleCellExperiment(list(counts=singlet.counts))
    print("Cells after removing doublets")
    print(ncol(singlet.counts))
    ###### SAVE CELL INFORMATION ######
    # Creating dataframe of umis per cell
    depths = Matrix::colSums(counts(sce_singlet))
    numi_df = data.frame("cell_id"=mtx@colData@listData$Barcode[cells_called][singlets],"nUMIs"=depths)
    # Add replicate info for when samples are eventually combined
    numi_df$repli = rep(replicate,nrow(numi_df))
    # Add tissue or region info for when samples are eventually combined
    numi_df$region = rep(tissue,nrow(numi_df))
    ###### NORMALIZE ######
    # Run scran normalization
    clusters <- quickCluster(sce_singlet, min.size=100)
    sce_singlet <- computeSumFactors(sce_singlet, cluster=clusters)
    sce_singlet = normalize(sce_singlet)
    normalized_logcounts = as.matrix(sce_singlet@assays@data[[2]])
    ###### EXPORT DATA ######
    norm_counts_df = as.data.frame(t(normalized_logcounts))
    numi_df_norm = unite(numi_df, "full_cell_id",cell_id,repli,region, sep = "_", remove = FALSE)
    rownames(norm_counts_df) = numi_df_norm$full_cell_id
    colnames(norm_counts_df) = rownames(counts(mtx))
    fwrite(norm_counts_df,paste(outdir,name,"_normalized_counts.csv",sep=""), row.names=TRUE)
    fwrite(numi_df_norm,paste(outdir,name,"_numis.csv",sep=""))
}

In [24]:
file = "/ihome/lbyrne/mej85/pgtb_human_retina_explant/analysis/cellranger/LB1_BYR819A1/outs/raw_feature_bc_matrix/"
name = "macula1"
replicate = "1"
tissue = "macula"
outdir = "/ihome/lbyrne/mej85/pgtb_human_retina_explant/analysis/"

In [25]:
suppressWarnings(run_qc(file,name,replicate,tissue,outdir))

[1] "Starting droplet count"
[1] 1193183
[1] "Cells called after removing empty droplets"
[1] 1716
[14:21:59] WARNING: amalgamation/../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[1] "Cells after removing doublets"
[1] 1505


In [21]:
file = "/ihome/lbyrne/mej85/pgtb_human_retina_explant/analysis/cellranger/LB4_BYR819A4/outs/raw_feature_bc_matrix/"
name = "macula2"
replicate = "2"
tissue = "macula"
outdir = "/ihome/lbyrne/mej85/pgtb_human_retina_explant/analysis/"

In [23]:
suppressWarnings(run_qc(file,name,replicate,tissue,outdir))

[1] "Starting droplet count"
[1] 1732316
[1] "Cells called after removing empty droplets"
[1] 3770
[14:15:56] WARNING: amalgamation/../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[1] "Cells after removing doublets"
[1] 3309
